In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

def scrape_details(driver):
    try:
        detail_title = driver.find_element(By.ID, 'topTitle').text.strip()
        detail_desc = driver.find_element(By.XPATH, '//*[@id="topTitle"]/following-sibling::p').text.strip()
        return detail_title, detail_desc
    except Exception as e:
        print("Error scraping details: ", e)
        return "N/A", "N/A"

def scrape_data(driver, start_url, total_pages=63):
    driver.get(start_url)
    time.sleep(2)  # 페이지 로딩 대기
    more_button = driver.find_element(By.XPATH, "//a[@title='여행정보 페이지로 이동']")
    more_button.click()
    time.sleep(2)  # '더보기' 후 로딩 대기

    results = []

    for i in range(1, total_pages + 1):
        print(f"Processing page {i}")
        items = driver.find_elements(By.CSS_SELECTOR, '.search_info_list .common_list li .cont')
        for idx in range(len(items)):
            try:
                items = driver.find_elements(By.CSS_SELECTOR, '.search_info_list .common_list li .cont')
                link = items[idx].find_element(By.CSS_SELECTOR, '.tit a')
                driver.execute_script("arguments[0].click();", link)
                time.sleep(2)  # 세부 페이지 로딩 대기

                title, summary = scrape_details(driver)
                results.append([title, summary])

                driver.back()
                time.sleep(2)  # 목록 페이지 로딩 대기
            except Exception as e:
                print("Error processing an item: ", e)

        # 페이지 네비게이션
        if i % 5 == 0 and i < total_pages:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.btn_next.ico')
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(5)  # 페이지 로딩 대기
        elif i < total_pages:
            # 다음 페이지 번호 클릭
            next_page = driver.find_element(By.XPATH, f"//a[@id='{i + 1}']")
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(5)  # 페이지 로딩 대기

    return results

# WebDriver 설정
driver = webdriver.Chrome()

url = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword=드라이브'
data = scrape_data(driver, url, 63)
driver.quit()

# 데이터프레임 생성 및 CSV 저장
df = pd.DataFrame(data, columns=['Title', 'Summary'])
df.to_csv('final.csv', index=False, encoding='utf-8-sig')
print("final.csv.")
